In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import scipy
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
%matplotlib inline
import pickle
pd.options.display.max_columns = 200

In [2]:
SEED=411 
kf = KFold(5, shuffle=True, random_state=SEED)

In [3]:
target = 'deal_probability'
y_train = pd.read_csv('train.csv', usecols=['activation_date', 'deal_probability'], parse_dates=['activation_date']) \
            .sort_values('activation_date') \
            .reset_index(drop=True).deal_probability.values
y_train

array([0.    , 0.    , 0.    , ..., 0.7376, 0.    , 0.7376])

In [4]:
#y_train = np.where(y_train > 0, 1, 0)

In [5]:
plants = pd.read_csv('all_features_plantsgo.csv.zip')

In [6]:
print(plants.shape)
plants

(2011862, 153)


,price,image_top_1,item_seq_number,activation_weekday,avg_days_up_user,avg_times_up_user,n_user_items,"cat_features_user_id_category_name_Одежда, обувь, аксессуары",cat_features_user_id_category_name_Детская одежда и обувь,cat_features_user_id_category_name_Товары для детей и игрушки,cat_features_user_id_category_name_Предложение услуг,cat_features_user_id_category_name_Квартиры,cat_features_user_id_category_name_Ремонт и строительство,cat_features_user_id_category_name_Мебель и интерьер,cat_features_user_id_category_name_Телефоны,cat_features_user_id_category_name_Коллекционирование,cat_features_user_id_category_name_Автомобили,"cat_features_user_id_category_name_Дома, дачи, коттеджи",cat_features_user_id_category_name_Товары для компьютера,cat_features_user_id_category_name_Аудио и видео,cat_features_user_id_category_name_Красота и здоровье,cat_features_user_id_category_name_Спорт и отдых,cat_features_user_id_category_name_Бытовая техника,cat_features_user_id_category_name_Часы и украшения,cat_features_user_id_category_name_Книги и журналы,cat_features_user_id_category_name_Оборудование для бизнеса,cat_features_user_id_category_name_Земельные участки,cat_features_user_id_category_name_Посуда и товары для кухни,cat_features_user_id_category_name_Товары для животных,cat_features_user_id_category_name_Фототехника,"cat_features_user_id_category_name_Игры, приставки и программы",cat_features_user_id_category_name_Грузовики и спецтехника,cat_features_user_id_category_name_Коммерческая недвижимость,cat_features_user_id_category_name_Оргтехника и расходники,cat_features_user_id_category_name_Растения,cat_features_user_id_category_name_Гаражи и машиноместа,cat_features_user_id_category_name_Планшеты и электронные книги,cat_features_user_id_category_name_Велосипеды,cat_features_user_id_category_name_Комнаты,cat_features_user_id_category_name_Музыкальные инструменты,cat_features_user_id_category_name_Собаки,cat_features_user_id_category_name_Ноутбуки,cat_features_user_id_category_name_Другие животные,cat_features_user_id_category_name_Охота и рыбалка,cat_features_user_id_category_name_Продукты питания,cat_features_user_id_category_name_Настольные компьютеры,cat_features_user_id_category_name_Кошки,cat_features_user_id_category_name_Аквариум,cat_features_user_id_category_name_Мотоциклы и мототехника,cat_features_user_id_category_name_Птицы,cat_features_user_id_category_name_Водный транспорт,cat_features_user_id_category_name_Готовый бизнес,cat_features_user_id_category_name_Билеты и путешествия,cat_features_user_id_category_name_Недвижимость за рубежом,price_pred,price_pred_onlydescription,price_pred_all,ridge_preds,ridge_preds_title,ridge_preds_description,ga,region,city,parent_category_name,category_name,user_type,param_1,param_2,param_3,title_tfidf_svd_1,title_tfidf_svd_2,title_tfidf_svd_3,title_tfidf_svd_4,title_tfidf_svd_5,description_tfidf_svd_1,description_tfidf_svd_2,description_tfidf_svd_3,description_tfidf_svd_4,description_tfidf_svd_5,region_mean_price,region_mean_image_top_1,region_mean_item_seq_number,region_mean_price_pred,region_mean_price_pred_all,region_mean_ridge_preds,city_mean_price,city_mean_image_top_1,city_mean_item_seq_number,city_mean_price_pred,city_mean_price_pred_all,city_mean_ridge_preds,parent_category_name_mean_price,parent_category_name_mean_image_top_1,parent_category_name_mean_item_seq_number,parent_category_name_mean_price_pred,parent_category_name_mean_price_pred_all,parent_category_name_mean_ridge_preds,category_name_mean_price,category_name_mean_image_top_1,category_name_mean_item_seq_number,category_name_mean_price_pred,category_name_mean_price_pred_all,category_name_mean_ridge_preds,user_type_mean_price,user_type_mean_image_top_1,user_type_mean_item_seq_number,user_type_mean_price_pred,user_type_mean_price_pred_all,user_type_mean_ridge_preds,param_1_mean_price,param_1_mean_image_top_1,param_1_mean_item_seq_number,param_1_mean_price_pred,param_1_mean_price_pred_all,param_1_mean_ridge_preds,param_2_

In [7]:
#plants.drop([col for col in plants.columns if 'ridge_preds' in col], axis=1, inplace=True)        
plants.columns.tolist()

['price',
 'image_top_1',
 'item_seq_number',
 'activation_weekday',
 'avg_days_up_user',
 'avg_times_up_user',
 'n_user_items',
 'cat_features_user_id_category_name_Одежда, обувь, аксессуары',
 'cat_features_user_id_category_name_Детская одежда и обувь',
 'cat_features_user_id_category_name_Товары для детей и игрушки',
 'cat_features_user_id_category_name_Предложение услуг',
 'cat_features_user_id_category_name_Квартиры',
 'cat_features_user_id_category_name_Ремонт и строительство',
 'cat_features_user_id_category_name_Мебель и интерьер',
 'cat_features_user_id_category_name_Телефоны',
 'cat_features_user_id_category_name_Коллекционирование',
 'cat_features_user_id_category_name_Автомобили',
 'cat_features_user_id_category_name_Дома, дачи, коттеджи',
 'cat_features_user_id_category_name_Товары для компьютера',
 'cat_features_user_id_category_name_Аудио и видео',
 'cat_features_user_id_category_name_Красота и здоровье',
 'cat_features_user_id_category_name_Спорт и отдых',
 'cat_feature

In [8]:
train_plants = plants.loc[:len(y_train)-1,:]
test_plants = plants.loc[len(y_train):,:].reset_index(drop=True)
del plants; gc.collect()

18

In [9]:
train_plants.shape, test_plants.shape

((1503424, 153), (508438, 153))

In [10]:
'''
origin = pd.read_csv('data/train.csv', usecols=['activation_date', 'item_id'], parse_dates=['activation_date'])
train_item_id = origin.sort_values('activation_date').reset_index(drop=True).item_id.values
del origin; gc.collect()

origin = pd.read_csv('data/test.csv', usecols=['item_id'])
test_item_id = origin.item_id.values
del origin; gc.collect()

train = pickle.load(open('train_selected_features', 'rb'))
test = pickle.load(open('test_selected_features', 'rb'))
train['item_id'] = train_item_id
test['item_id'] = test_item_id

train = train.merge(train_plants, on='item_id', how='left')
test = test.merge(test_plants, on='item_id', how='left')

train.drop('item_id', axis=1, inplace=True)
test.drop('item_id', axis=1, inplace=True)

del train_plants, test_plants; gc.collect()
'''

train = train_plants
test = test_plants

origin = pd.read_csv('train.csv', usecols=['activation_date', 'item_id'], parse_dates=['activation_date'])
train['activation_date'] = origin['activation_date']
train = train.sort_values('activation_date').reset_index(drop=True).drop('activation_date', axis=1)
train.drop('item_id', axis=1, inplace=True)
test.drop('item_id', axis=1, inplace=True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
train = pd.concat([train, pickle.load(open('train_selected_features', 'rb'))], axis=1)
test = pd.concat([test, pickle.load(open('test_selected_features', 'rb'))], axis=1)

In [12]:
print(train.shape)
train

(1503424, 213)


,price,image_top_1,item_seq_number,activation_weekday,avg_days_up_user,avg_times_up_user,n_user_items,"cat_features_user_id_category_name_Одежда, обувь, аксессуары",cat_features_user_id_category_name_Детская одежда и обувь,cat_features_user_id_category_name_Товары для детей и игрушки,cat_features_user_id_category_name_Предложение услуг,cat_features_user_id_category_name_Квартиры,cat_features_user_id_category_name_Ремонт и строительство,cat_features_user_id_category_name_Мебель и интерьер,cat_features_user_id_category_name_Телефоны,cat_features_user_id_category_name_Коллекционирование,cat_features_user_id_category_name_Автомобили,"cat_features_user_id_category_name_Дома, дачи, коттеджи",cat_features_user_id_category_name_Товары для компьютера,cat_features_user_id_category_name_Аудио и видео,cat_features_user_id_category_name_Красота и здоровье,cat_features_user_id_category_name_Спорт и отдых,cat_features_user_id_category_name_Бытовая техника,cat_features_user_id_category_name_Часы и украшения,cat_features_user_id_category_name_Книги и журналы,cat_features_user_id_category_name_Оборудование для бизнеса,cat_features_user_id_category_name_Земельные участки,cat_features_user_id_category_name_Посуда и товары для кухни,cat_features_user_id_category_name_Товары для животных,cat_features_user_id_category_name_Фототехника,"cat_features_user_id_category_name_Игры, приставки и программы",cat_features_user_id_category_name_Грузовики и спецтехника,cat_features_user_id_category_name_Коммерческая недвижимость,cat_features_user_id_category_name_Оргтехника и расходники,cat_features_user_id_category_name_Растения,cat_features_user_id_category_name_Гаражи и машиноместа,cat_features_user_id_category_name_Планшеты и электронные книги,cat_features_user_id_category_name_Велосипеды,cat_features_user_id_category_name_Комнаты,cat_features_user_id_category_name_Музыкальные инструменты,cat_features_user_id_category_name_Собаки,cat_features_user_id_category_name_Ноутбуки,cat_features_user_id_category_name_Другие животные,cat_features_user_id_category_name_Охота и рыбалка,cat_features_user_id_category_name_Продукты питания,cat_features_user_id_category_name_Настольные компьютеры,cat_features_user_id_category_name_Кошки,cat_features_user_id_category_name_Аквариум,cat_features_user_id_category_name_Мотоциклы и мототехника,cat_features_user_id_category_name_Птицы,cat_features_user_id_category_name_Водный транспорт,cat_features_user_id_category_name_Готовый бизнес,cat_features_user_id_category_name_Билеты и путешествия,cat_features_user_id_category_name_Недвижимость за рубежом,price_pred,price_pred_onlydescription,price_pred_all,ridge_preds,ridge_preds_title,ridge_preds_description,ga,region,city,parent_category_name,category_name,user_type,param_1,param_2,param_3,title_tfidf_svd_1,title_tfidf_svd_2,title_tfidf_svd_3,title_tfidf_svd_4,title_tfidf_svd_5,description_tfidf_svd_1,description_tfidf_svd_2,description_tfidf_svd_3,description_tfidf_svd_4,description_tfidf_svd_5,region_mean_price,region_mean_image_top_1,region_mean_item_seq_number,region_mean_price_pred,region_mean_price_pred_all,region_mean_ridge_preds,city_mean_price,city_mean_image_top_1,city_mean_item_seq_number,city_mean_price_pred,city_mean_price_pred_all,city_mean_ridge_preds,parent_category_name_mean_price,parent_category_name_mean_image_top_1,parent_category_name_mean_item_seq_number,parent_category_name_mean_price_pred,parent_category_name_mean_price_pred_all,parent_category_name_mean_ridge_preds,category_name_mean_price,category_name_mean_image_top_1,category_name_mean_item_seq_number,...,param_1_mean_price_pred_all,param_1_mean_ridge_preds,param_2_mean_price,param_2_mean_image_top_1,param_2_mean_item_seq_number,param_2_mean_price_pred,param_2_mean_price_pred_all,param_2_mean_ridge_preds,param_3_mean_price,param_3_mean_image_top_1,param_3_mean_item_seq_number,param_3_mean_price_pred,param_3_mean_price_pred_all,param_3_mean_ridge_preds,user_id_nunique_parent_category_name,user_id_nunique_categ

In [13]:
from GridSearcher import data_loader, model_loader, fit_params, get_oof_predictions
ml = model_loader('lgb')

In [ ]:
rounds = 24000
early_stop_rounds = 200
'''
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss,accuracy',
    'min_child_weight': 1.5,
    'num_leaves': 2 ** 5,
    'lambda_l2': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.5,
    'colsample_bylevel': 0.5,
    'learning_rate': 0.1,
    'seed': 2018,
    'nthread': 4,
    'silent': True,
}
'''
params = {
    'booster': 'gbtree',
    'objective': 'reg:logistic',
    'eval_metric': 'rmse',
    'min_child_weight': 1.5,
    'max_depth': 8,
    'reg_lambda': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.8,
    'eta': 0.1,
    'seed': 2018,
    'nthread': 8,
    'silent': True,
}

ret = np.zeros((train.shape[0],))
ret_test = np.zeros((test.shape[0],))
ret_models = []
    
dtest = xgb.DMatrix(test)
                     
for train_ix, val_ix in kf.split(train):
    dtrain = xgb.DMatrix(train.loc[train_ix,:], label=y_train[train_ix],)
    dvalid = xgb.DMatrix(train.loc[val_ix,:], label=y_train[val_ix],)
    
    model = xgb.train(params, dtrain, 
                      evals=[(dtrain, 'train'), (dvalid, 'valid')],
                      num_boost_round=rounds, 
                      early_stopping_rounds=early_stop_rounds, 
                      verbose_eval=100)
    
    ret[val_ix] = model.predict(dvalid)
    ret_test += model.predict(dtest)
    ret_models.append(model)
    
    del dtrain, dvalid
    
ret_test = ret_test / 5.

[0]	train-rmse:0.412369	valid-rmse:0.412354
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.211819	valid-rmse:0.216684
[200]	train-rmse:0.206144	valid-rmse:0.21529
[300]	train-rmse:0.202205	valid-rmse:0.214735
[400]	train-rmse:0.19875	valid-rmse:0.214494
[500]	train-rmse:0.195414	valid-rmse:0.214394
[600]	train-rmse:0.192178	valid-rmse:0.214323
[700]	train-rmse:0.189041	valid-rmse:0.21429
[800]	train-rmse:0.185826	valid-rmse:0.214273


In [ ]:
for model in ret_models:
    fig, ax = plt.subplots(figsize=(10, 14))
    xgb.plot_importance(model, max_num_features=100, ax=ax)
    plt.title("XGBOOST Feature Importance")
    plt.show()

In [ ]:
prefix = 'baseline_xgb'

In [ ]:
pd.DataFrame(data=ret, columns=[prefix+'_pred']).to_csv(prefix+'_oof_val_pred.csv', index=False)
pd.DataFrame(data=ret_test, columns=[prefix+'_pred']).to_csv(prefix+'_oof_test_pred.csv', index=False)

In [ ]:
subm = pd.read_csv('sample_submission.csv')
subm['deal_probability'] = np.clip(ret_test, 0, 1)
subm.to_csv(prefix+'_submission.csv', index=False)

In [ ]:
# baseline: cv:  lb: